# Issues:
- AudioDescriptors class takes ~20 hours to compute
- AudioDescriptorsExtended a bit less but still too long

In [1]:
!pip install essentia-tensorflow
!pip install numpy pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 15.9 MB/s eta 0:00:0000:0100:01
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

from google.colab import auth
auth.authenticate_user()

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import json

import essentia.standard as es
import pandas as pd

import IPython.display as ipd
from tqdm import tqdm

from itertools import chain

2023-02-21 07:57:59.109284: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [89]:
WORKING_DIR = Path(os.getcwd())
MODELS_HOME = Path(os.getcwd(), 'models')
DATASET_PATH = Path('/Users/francescopapaleo/GDrive/essentia-playlist/MusAV/audio_chunks/')

!cd "$DATASET_PATH" && ls

audio.000 audio.001 audio.002 audio.003 audio.004 audio.005 audio.006


In [36]:
! mkdir models
! curl -L -o models/voice_instrumental-musicnn-mtt-2.pb "https://essentia.upf.edu/models/classifiers/voice_instrumental/voice_instrumental-musicnn-mtt-2.pb"
! curl -L -o models/discogs-effnet-bs64-1.pb "https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.pb"
! curl -L -o models/msd-musicnn-1.pb "https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.pb"
! curl -L -o models/emomusic-musicnn-msd-2.pb "https://essentia.upf.edu/models/classification-heads/emomusic/emomusic-musicnn-msd-2.pb"
! curl -L -o models/labels.py "https://raw.githubusercontent.com/MTG/essentia-replicate-demos/main/effnet-discogs/labels.py"


mkdir: models: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3163k  100 3163k    0     0  3387k      0 --:--:-- --:--:-- --:--:-- 3383k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.5M  100 17.5M    0     0  4937k      0  0:00:03  0:00:02  0:00:01 5345k     0  0:00:03  0:00:03 --:--:-- 4937k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3123k  100 3123k    0     0  2570k      0  0:00:01  0:00:01 --:--:-- 2570k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 82460  100 82460    0     0   619k      0 --:--:-- --:--:-- -

In [79]:
# function to get a list of all audio files to be analysed under a folder 

def get_all_files(dir_to_analyse):
    names_rel_list = []
    for (dirpath, dirnames, filenames) in os.walk(dir_to_analyse):
        for f in filenames:
            tmp_rel_path = os.path.relpath(dirpath, dir_to_analyse)
            tmp_file_path = os.path.join(tmp_rel_path, f)
            names_rel_list.append(tmp_file_path)
    return names_rel_list

1944


In [90]:
# Check if the files are already listed, if not calling the function

if not os.path.exists('data/all_files_list.json'):
    print('Listing all files in the', DATASET_PATH)
    with open('data/all_files_list.json', 'w') as f:
        all_files_list = []
        all_files_list = get_all_files(DATASET_PATH)
        json.dump(all_files_list, f)
    print("File list saved to all_files_list.json")
    print("The list contains", len(all_files_list), "files")
else:
    print("File list exists, you can run one of the descriptors clases")
    with open('data/all_files_list.json', 'r') as f:
        all_files_list = json.load(f)
    print("The list contains", len(all_files_list), "files")

Listing all files in the /Users/francescopapaleo/GDrive/essentia-playlist/MusAV/audio_chunks
File list saved to all_files_list.json
The list contains 2100 files


In [51]:
if not os.path.exists(os.path.join(GIT_DATA, "all_files_list.json")):
    print("Listing all files in the DATASET_PATH...")
    with open(GIT_DATA / "all_files_list.json", "w") as f:
        all_files_list = get_all_files(DATASET_PATH)
        json.dump(all_files_list, f)
    print("File list saved to all_files_list.json")
    print("The list contains", len(all_files_list), "files")
else:
    print("File list exists, you can run one of the descriptors clases")
    # Load the JSON if already stored
    with open(GIT_DATA / "all_files_list.json", 'r') as f:
        all_files_list = json.load(f)
    print("The list contains", len(all_files_list), "files")

if not all_files_list:
    print("No audio files found in the specified directory and its subdirectories")


Listing all files in the DATASET_PATH...


0it [00:00, ?it/s]

File list saved to all_files_list.json
The list contains 0 files
No audio files found in the specified directory and its subdirectories


In [15]:
descriptors = AudioDescriptors()

with open("descriptors_output.json", "w") as f:
    for file in tqdm(all_files_list):
        descriptors_dict = {}

        # Compute the audio features for the current file
        audio = descriptors.audio_16(file)
        tempo, danceability = descriptors.tempo_dance(file)
        style_label = descriptors.style_ml(audio)
        vi_label = descriptors.vi_ml(audio)
        valence, arousal = descriptors.av_ml(audio)

        descriptors_dict['file'] = file
        descriptors_dict['tempo'] = float(tempo)
        descriptors_dict['danceability'] = float(danceability)
        descriptors_dict['valence'] = float(valence)
        descriptors_dict['arousal'] = float(arousal)
        descriptors_dict['style'] = style_label
        descriptors_dict['vi_label'] = vi_label

        json.dump(descriptors_dict, f)
        f.write('\n')

NameError: name 'AudioDescriptors' is not defined

In [26]:
### CLASS to compute audio descriptors WITH STYLE ACTIVATIONS for 400 STYLES ### 


# voice_instrumental-msd-musicnn-1.pb

class AudioDescriptorsExtended:
    def __init__(self):
        self.model_effnet = es.TensorflowPredictEffnetDiscogs(graphFilename='/Users/francescopapaleo/Dropbox/Mac/Documents/git-box/streamlit/models/discogs-effnet-bs64-1.pb')
        self.model_vi = es.TensorflowPredictMusiCNN(graphFilename='/Users/francescopapaleo/Dropbox/Mac/Documents/git-box/streamlit/models/voice_instrumental-musicnn-mtt-2.pb', output='model/dense/BiasAdd')
        self.model_av_emb = es.TensorflowPredictMusiCNN(graphFilename="/Users/francescopapaleo/Dropbox/Mac/Documents/git-box/streamlit/models/msd-musicnn-1.pb", output='model/dense/BiasAdd')
        self.model_av = es.TensorflowPredict2D(graphFilename='/Users/francescopapaleo/Dropbox/Mac/Documents/git-box/streamlit/models/emomusic-musicnn-msd-2.pb', output='model/Identity')

    def tempo_dance(self, path_to_file):
        audio = es.MonoLoader(filename=path_to_file, sampleRate=44100)()
        bpm, beats, beats_confidence, _, beats_intervals = es.RhythmExtractor2013()(audio)
        danceability, dfa = es.Danceability()(audio)
        return bpm, danceability

    def audio_16(self, path_to_file):
        audio_load_16 = es.MonoLoader(filename=path_to_file, sampleRate=16000)()
        return audio_load_16

    def style_ml(self, audio_load_16):
        activations = self.model_effnet(audio_load_16)
        activations_mean = np.mean(activations, axis=0)
        activations_list = list(activations_mean.astype(float))
        return activations_list


    def vi_ml(self, audio_load_16):
        activations = self.model_vi(audio_load_16)
        v_i_mean = np.mean(activations, axis=0, keepdims=True)[0]
        vi_tmp = (v_i_mean + 1) / 2  # Scale to range [0, 1]
        vi_scaled = vi_tmp.tolist()
        return vi_scaled

    def av_ml(self, audio_load_16):
        embeddings = self.model_av_emb(audio_load_16)
        activations = self.model_av(embeddings)
        activations_mean = np.mean(activations, axis=0, keepdims=True)[0]
        valence = activations_mean[0]
        arousal = activations_mean[1]
        return valence, arousal

    def compute_descriptors(self, files_list):
        all_descriptors = []
        for file_path in files_list:
            rel_path = os.path.relpath(file_path)
            audio_16 = self.audio_16(file_path)
            activations_list = self.style_ml(audio_16)
            vi_scaled = self.vi_ml(audio_16)
            valence, arousal = self.av_ml(audio_16)
            bpm, danceability = self.tempo_dance(file_path)

            descriptor_dict = {
                'file_path': rel_path,
                'bpm': str(bpm),
                'danceability': str(danceability),
                'style_activations': str(activations_list),
                'vi_scaled': str(vi_scaled),
                'valence': str(valence),
                'arousal': str(arousal)
            }
            all_descriptors.append(descriptor_dict)

        return all_descriptors

In [27]:
extended_descriptors = AudioDescriptorsExtended()

with open("extended_descriptors_output.json", "w") as f:
    for file_path in tqdm(all_files_list):
        features = extended_descriptors.compute_descriptors([file_path])
        json.dump(features, f, indent=1)

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `/Users/francescopapaleo/Dropbox/Mac/Documents/git-box/streamlit/models/discogs-effnet-bs64-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `/Users/francescopapaleo/Dropbox/Mac/Documents/git-box/streamlit/models/voice_instrumental-musicnn-mtt-2.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `/Users/francescopapaleo/Dropbox/Mac/Documents/git-box/streamlit/models/msd-musicnn-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `/Users/francescopapaleo/Dropbox/Mac/Documents/git-box/streamlit/models/emomusic-musicnn-msd-2.pb`
  0%|          | 0/2100 [00:00<?, ?it/s]


RuntimeError: Error while configuring MonoLoader: AudioLoader: Could not open file "/content/gdrive/MyDrive/AMPLAB/MusAV/audio_chunks/audio.000/1t/1tnci80rcW5LjqQqBwdZsW.mp3", error = No such file or directory

In [19]:
print(all_files_list[0])

IndexError: list index out of range